In [1]:
!pip install mtcnn opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.4 MB/s eta 0:00:00


In [2]:
import gdown

# Download pretrained model from official repo
url = "https://github.com/oarriaga/face_classification/raw/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5"
gdown.download(url, "mini_xception_engaged_confused.h5", quiet=False)


Downloading...
From: https://github.com/oarriaga/face_classification/raw/master/trained_models/emotion_models/fer2013_mini_XCEPTION.102-0.66.hdf5
To: /content/mini_xception_engaged_confused.h5
100%|██████████| 873k/873k [00:00<00:00, 6.50MB/s]


'mini_xception_engaged_confused.h5'

In [6]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from IPython.display import HTML
from base64 import b64encode

# Load model and face detector
model = load_model("mini_xception_engaged_confused.h5", compile=False)

detector = MTCNN()

# FER2013 emotion classes
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

def map_emotion(emotion):
    if emotion in ['Happy', 'Surprise']:
        return 'Engaged'
    elif emotion in ['Angry', 'Disgust', 'Fear', 'Sad']:
        return 'Confused'
    else:
        return 'Neutral'

# Load video from Google Drive
video_path = '/content/drive/MyDrive/clarifai/input/input_video.mp4'
cap = cv2.VideoCapture(video_path)

# Prepare output video path
out_name = "/content/output_emotion.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(out_name, fourcc, fps, (w, h))

# Frame processing
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(rgb)

    for face in faces:
        x, y, w_, h_ = face['box']
        x, y = max(0, x), max(0, y)
        face_img = rgb[y:y+h_, x:x+w_]
        face_img = cv2.resize(face_img, (64, 64))
        gray_face = cv2.cvtColor(face_img, cv2.COLOR_RGB2GRAY)
        roi = gray_face.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)

        preds = model.predict(roi, verbose=0)[0]
        label = emotion_labels[preds.argmax()]
        engagement = map_emotion(label)

        color = (0, 255, 0) if engagement == 'Engaged' else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x + w_, y + h_), color, 2)
        cv2.putText(frame, engagement, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    out.write(frame)

cap.release()
out.release()
